In [ ]:
import numpy as np
import random
from qiskit import QuantumCircuit, BasicAer
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt
import numpy as np
import random
import math
import copy
from copy import deepcopy
import itertools
simulator = BasicAer.get_backend('qasm_simulator')
from qiskit import *
from qiskit.visualization import plot_histogram
from qiskit.providers.aer.noise import pauli_error
from collections import defaultdict
from qiskit import Aer
import numpy as np
import matplotlib.pyplot as plt
import qiskit.quantum_info as qi

import scipy.linalg as la

import import_ipynb
from circuit_evolution import *

In [ ]:
from pyqubo import Spin, Binary

def ising_model_partitioning(number_set):
    """
    Args:
    1) A matrix of the distances (upper triangular)
    2) Vector of the times
    3) Number of bits in the floating-point approximation
    4) If ising = True, the output is an ising model

    Output:
    A linear system problem written as a qubo formulation

    """    
    #set partitioning as a qubo
    qubo = 0
    for k, n in enumerate(number_set):
        qubo += n*(2*Binary("s"+str(k+1)) - 1)
    qubo = qubo**2


    model = qubo.compile()    
    linear, quadratic, offset = model.to_ising()
    
    return linear, quadratic, offset


# problem
numbers = [5,5,1,1,1,1,1,1,1,3]
linear, quadratic, offset = ising_model_partitioning(numbers)

#Hamiltonian construction
num_variables=len(numbers)
H=[[0 for i in range(2**num_variables)] for j in range(2**num_variables) ] #hamiltonian initialize

#quadratic terms from ising model
for key, value in quadratic.items():
    q=QuantumRegister(num_variables,'q')
    circuit = QuantumCircuit(q)
    circuit.z(int(key[0][1:])-1)
    circuit.z(int(key[1][1:])-1)    
    op = qi.Operator(circuit)
    array=op.data
    H+=value*array    

#linear terms from ising model
for key, value in linear.items():
    q=QuantumRegister(num_variables,'q')
    circuit = QuantumCircuit(q)
    circuit.z(int(key[1:])-1)
    op = qi.Operator(circuit)
    array=op.data
    H+=value*array

In [ ]:
results = la.eig(H) #Expected minimum energy
print(min(results[0]))

In [ ]:
func_val=get_min_circuit(H) #gives the value of optimization function at each generation

In [ ]:
func_val[-1][0] #gives the value of target function on last generation

In [ ]:
final_circuit=get_circuit(func_val[-1][1])

In [ ]:
qc = QuantumCircuit(num_variables, num_variables)
qc = qc.compose(final_circuit)
qc.measure(range(num_variables), range(num_variables))

counts = execute(qc, backend=simulator, shots=1000).result().get_counts(qc)

In [ ]:
final_circuit=get_circuit(func_val[-1][1])
final_circuit.draw(initial_state=True, output="mpl", fold=-1)

In [ ]:
values3=[]
for i in func_val:
    values3.append(i[0].real)
    
plt.plot(values3, linewidth=2, color='red')
plt.xlabel('No. of generations')
plt.ylabel('Energy Value')
plt.title('Seismic inversion - Ising model')
plt.grid()
plt.show()

In [ ]:
counts

In [ ]:
import csv
import pandas as pd

easy_set = pd.read_csv(r'C:/Users/OS-5818/OneDrive/Documentos/Anton/bloot/cilamce_2022/data/Set_1000_easy_03.txt')
hard_set = pd.read_csv(r'C:/Users/OS-5818/OneDrive/Documentos/Anton/bloot/cilamce_2022/data/Set_1000_Hard_095.txt')
medium_set = pd.read_csv(r'C:/Users/OS-5818/OneDrive/Documentos/Anton/bloot/cilamce_2022/data/Set_1000_Medium_05.txt')